In [66]:
# pre reqs 
"""
 zt = 2xt/vsound 
 ut = xt.. 
 xt = xt0 + ut(t)^2/2 
 ept = N(0,R)
 delt = N(0,Q) 
 X = [xt,xt.] 
 
 326 time stamps with 0.01 interval 
 xt(0)= 0 , var = 1e-4I 
 Q = 0.01 
 var(x) = 0.1^2 , var(x.) = 0.5^2 , R is diag 
 vsound = 3000 km/hr 
 u(t) = ... 
"""
import numpy as np 
import math 
tol = 1e-12

In [67]:
# True x,v 
def u(t):
    if t < 0.25:
        return 400 
    elif t > 3 and t < 3.25:
        return -400 
    else:
        return 0 

def v(t):
    if t < 0.25: 
        return u(t)*t 
    elif t > 3 and t < 3.25:
        return 100+u(t)*(t-3)
    else:
        return 0.25*u(0) 

def x(t):
    if t < 0.25:
        return (1/2)*u(t)*t**2 
    elif t > 3 and t <= 3.25:
        return (1/2)*u(0)*(0.25)**2 + (2.75)*v(0.25) + v(0.25)*(t-3) + (1/2)*u(t)*(t-3)**2
    else:
        return (1/2)*u(0)*(0.25)**2 + (t-0.25)*v(0.25)


In [68]:
# estimated x,v 
sig_x = 0.1 
sig_v = 0.5
sig_s = 0.01
time_step = 0.01
N = 326
v_sound = 3000
time = [(0.01*i-tol) for i in range(0,326)]

mu_0 = np.transpose(np.array([0,0]))
mu_0 = mu_0[:,np.newaxis]
sigma_0 = 1e-4*np.array([[1,0],[0,1]])
 
R = np.array([[sig_x**2,0],[0,sig_v**2]])
Q = np.array([[sig_s**2]])

A_t = np.array([[1,time_step],[0,1]]) 
B_t = np.transpose(np.array([0.5*time_step**2,time_step]))
B_t = B_t[:,np.newaxis]

C_t = np.array([[2/v_sound,0]]) 


def kalman_filter(mu_prev,sigma_prev,u_t,z_t):

    mu_t = A_t@mu_prev + B_t@u_t 
    sigma_t = A_t@sigma_prev + R 
    
    K_t = sigma_t@np.transpose(C_t)@np.linalg.inv((C_t@sigma_t@np.transpose(C_t)+Q))
    mu_t = mu_t + K_t@(z_t-C_t@mu_t)
    sigma_t = (np.array([[1,0],[0,1]])-K_t@C_t)@sigma_t
    return mu_t,sigma_t,K_t

from numpy.random import multivariate_normal 
print(mu_0)
print(np.transpose(mu_0))
state_t = [np.transpose(np.array(multivariate_normal(np.transpose(mu_0)[0],sigma_0,1)))]
for t in time[1:]:
    noise_sample = np.transpose(np.array(multivariate_normal(np.array([0,0]),R,1)[0]))
    noise_sample = noise_sample[:,np.newaxis]
    state_t.append(A_t@state_t[-1]+B_t@np.array([[u(t)]])+noise_sample)
  

z_t = [(C_t@state_t[i]+multivariate_normal([0],Q,1)[0]) for i in range(len(time))]
K_t = []
mu_t = [mu_0]
sigma_t = [sigma_0]


for i,t in enumerate(time):
    mu,sigma,K = kalman_filter(mu_t[-1],sigma_t[-1],np.array([[u(t)]]),z_t[i]) 
    K_t.append(K)
    mu_t.append(mu)
    sigma_t.append(sigma) 



    
    
x_est = [mu[0][0] for mu in mu_t]
v_est = [mu[1][0] for mu in mu_t]   
x_est = x_est[:-1]
v_est= v_est[:-1] 
x_true = [state_t[i][0][0] for i in range(len(time))]
v_true = [state_t[i][1][0] for i in range(len(time))]
# print(x_true)
print(x_est)
print(x_true)
diff = [x_est[i]-state_t[i][0] for i in range(len(time))]
print(diff)
    
    
    


[[0]
 [0]]
[[0 0]]
[np.int64(0), np.float64(0.019971547714097728), np.float64(0.07982734133440983), np.float64(0.17913198018892562), np.float64(0.3165386776007145), np.float64(0.49743684657723014), np.float64(0.7165486591000556), np.float64(0.9776239087730505), np.float64(1.2825865362879778), np.float64(1.6266025188407756), np.float64(2.0062954049185326), np.float64(2.4330182065494528), np.float64(2.8857460045011174), np.float64(3.393423091883782), np.float64(3.927745818449974), np.float64(4.511801896448669), np.float64(5.12489491216633), np.float64(5.8098462861385025), np.float64(6.494072797360416), np.float64(7.232928376029798), np.float64(8.003681493792383), np.float64(8.826839983452222), np.float64(9.668899204697246), np.float64(10.565380524877394), np.float64(11.497655524954762), np.float64(12.451264457399638), np.float64(13.464922979018048), np.float64(14.507459706634569), np.float64(15.58153533773996), np.float64(16.61381360574821), np.float64(17.620327049129262), np.float64(18.

In [69]:
import plotly.express as px
print(len(x_true))
print(len(time))
fig = px.scatter(x=time, y=x_true)
fig.add_scatter(x=time,y=v_true,marker=dict(color='blue'))
fig.add_scatter(x=time,y=x_est,marker=dict(color='green'))
fig.add_scatter(x=time,y=v_est,marker=dict(color='green'))
fig.show()
fig.write_html("r.html")

326
326


In [70]:
print(len(time))
print(len(x_est))
fig2 = px.scatter(x=time,y=x_est)
fig2.show()

326
326
